In [25]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt
data_dir='Airforce_with_score.csv'

In [26]:
data=pd.read_csv(data_dir,encoding='cp949')

In [27]:
data

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,total_price,talk_user_count,first_talk_date,sell_count,register_prod_count,final_purchase_date,purchase_duration,talk_duration,des_length,sum_score
0,0bc912bd886998ebc4c0c5ba6b362700,3,2024-04-05 13:57:00,나이키 에어포스,"나이키,에어포스,275",75000,20,1,0,0,...,87620.0,2.0,2024-04-05,1,1,2024-04-05 16:53:00,2.933333,-1,59,5.206606
1,ac78a060c12396c566e9eb7de105243c,3,2024-01-22 16:52:00,나이키 에어포스1 올백 275 팝니다,NaN,60000,21,1,0,0,...,60000.0,1.0,2024-01-22,3,1,2024-01-31 17:10:00,216.300000,-1,62,9.941372
2,61b88b1a528f28b4032310703a2b1dc7,3,2024-02-14 15:34:00,나이키 에어포스 1 올백 240,"나이키,에어포스,에어포스1,나이키포스",69000,21,0,0,0,...,69000.0,5.0,2024-02-15,2,1,2024-03-05 19:51:00,484.283333,0,32,-11.412334
3,9e39d945f0fcc4ec11dce688e2d4b28f,3,2024-03-24 21:01:00,나이키 에어포스1 올백 올흰 240,NaN,35000,21,0,0,0,...,36800.0,1.0,2024-03-26,4,1,2024-04-06 18:51:00,309.833333,1,138,-16.264341
4,c6be71e80bd2b2d9cee48eedc2658c64,3,2023-12-11 12:11:00,250]나이키 에어포스 1 07 화이트,"나이키에어포스1,에어포스250",70000,21,0,0,0,...,73000.0,5.0,2024-01-18,855,1,2024-03-25 11:18:00,2519.116667,37,187,-1.524808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,1d4265d3b20a3ea825a2940f56460810,3,2024-04-28 22:58:00,나이키 에어포스1 220,NaN,80000,20,1,0,0,...,82800.0,1.0,2024-05-04,22,1,2024-05-04 12:28:00,133.500000,5,125,6.196063
123,24a612a7fd794fbe355837467dbc623c,3,2023-12-17 11:55:00,나이키 에어포스 1 07 로우 275,NaN,70000,21,0,0,0,...,74000.0,2.0,2023-12-18,2,1,2023-12-23 14:08:00,146.216667,0,200,1.627951
124,01b7f8442fd16cea51124d129762545c,3,2024-04-17 11:24:00,에어포스1 올백 280,"나이키에어포스,나이키에어포스1,에어포스280,에어포스1",70000,21,0,0,0,...,73000.0,2.0,2024-04-21,2,1,2024-04-29 17:51:00,294.450000,3,49,9.237624
125,801a94f2ac6be402d6b81cbb574aea81,3,2024-04-20 14:55:00,나이키 에어포스1 로우 올백 화이트 275,NaN,50000,21,0,0,0,...,54100.0,7.0,2024-04-20,45,1,2024-05-01 14:51:00,263.933333,-1,39,-13.804903


In [36]:
#Translate
import googletrans

translator= googletrans.Translator()

def translate_text(text, src='ko', dest='en'):
    try:
        return translator.translate(text, src=src, dest=dest).text
    except Exception as e:
        return str(e)
data['translated_desc'] = data['description'].apply(lambda x: translate_text(x))


In [37]:
data.to_csv("Airforce_with_score_translated.csv",index=False,encoding='cp949')

## 영어 불용어 제거

In [50]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# 불용어 리스트 로드
stop_words = set(stopwords.words('english'))
# 구두점 리스트 로드
punctuations = set(string.punctuation)

# 불용어 및 구두점 제거 함수 정의
def remove_stopwords_and_punctuations(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words and word not in punctuations]
    return ' '.join(filtered_text)

data['cleaned_translated_desc'] = data['translated_desc'].apply(lambda x: remove_stopwords_and_punctuations(x))


In [42]:
data.to_csv("Airforce_with_score_translated.csv",index=False,encoding='cp949')

In [51]:
#한글 불용어 제거
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return stopwords

def preprocessing(description, remove_stopwords =False):
    description_text=BeautifulSoup(description,'html5lib').get_text()
    description_text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', description_text)
    description_text = re.sub(r'\s+', ' ', description_text).strip()
    okt = Okt()
    description_text = description_text.replace('사 용감', '사용감')
    description_text = description_text.replace('사용 감', '사용감')
    word_description=okt.morphs(description_text, stem=False) 

    if remove_stopwords:
        korean_stop_words = load_stopwords('stopwords-ko.txt')
        word_description=[token for token in word_description if not token in korean_stop_words]
    return word_description


In [52]:
data['cleaned_description'] = data['description'].apply(lambda x: preprocessing(x, remove_stopwords=True))
data['des_length'] = data['cleaned_translated_desc'].apply(len)

C:\Users\sunwoong\AppData\Local\Temp\ipykernel_37524\1937951895.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description_text=BeautifulSoup(description,'html5lib').get_text()


In [53]:
data.to_csv("Airforce_with_score_translated.csv",index=False,encoding='cp949')

In [54]:
data=pd.read_csv('Airforce_with_score_translated.csv',encoding='cp949')

In [55]:
data['cleaned_translated_desc'] = data['cleaned_translated_desc'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)


# 'text' 컬럼의 모든 문자열을 하나의 문자열로 결합
all_text = ' '.join(data['cleaned_translated_desc'])

# 문자열을 공백을 기준으로 나누어 단어 리스트 생성
words = all_text.split()

# 단어 빈도수 계산
word_counts = Counter(words)

# 상위 50개 가장 많이 나온 단어 찾기
top_50_common_words = word_counts.most_common(30)

print("상위 30개 가장 많이 나온 단어:")
for word, count in top_50_common_words:
    print(f"단어: '{word}', 빈도수: {count}")

상위 30개 가장 많이 나온 단어:
단어: 'Nike', 빈도수: 43
단어: 'product', 빈도수: 39
단어: 'Force', 빈도수: 38
단어: 'Air', 빈도수: 35
단어: ''s', 빈도수: 34
단어: 'size', 빈도수: 31
단어: 'n't', 빈도수: 29
단어: 'courier', 빈도수: 28
단어: '1', 빈도수: 27
단어: 'Please', 빈도수: 27
단어: 'box', 빈도수: 26
단어: 'new', 빈도수: 23
단어: 'bought', 빈도수: 22
단어: 'x', 빈도수: 20
단어: 'times', 빈도수: 19
단어: 'Size', 빈도수: 19
단어: 'clean', 빈도수: 19
단어: 'wear', 빈도수: 18
단어: 'cream', 빈도수: 15
단어: 'shoes', 빈도수: 15
단어: 'good', 빈도수: 15
단어: 'White', 빈도수: 14
단어: 'contact', 빈도수: 14
단어: 'direct', 빈도수: 14
단어: 'picture', 빈도수: 13
단어: 'exchange', 빈도수: 13
단어: 'Courier', 빈도수: 13
단어: 'price', 빈도수: 12
단어: '07', 빈도수: 12
단어: 'white', 빈도수: 11


In [57]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# 텍스트 토큰화
sentences = data['cleaned_translated_desc'].apply(word_tokenize).tolist()

# Word2Vec 모델 훈련
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 단어 벡터 확인
word_vectors = model.wv

In [59]:
def calculate_similarity(row):
    words = word_tokenize(row['cleaned_translated_desc'])
    if len(words) < 2:
        return 0
    else:
        return word_vectors.n_similarity(words[:-1], words[1:])

data['similarity_score'] = data.apply(calculate_similarity, axis=1)


In [65]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data['standardized_similarity_score'] = scaler.fit_transform(data[['similarity_score']])


In [67]:
data.to_csv("Airforce_with_score_translated.csv",index=False,encoding='cp949')

In [66]:
data=data[['price','flag_used','flag_taekpo','image_count','talk_user_count','talk_duration','des_length',
           'sum_score','purchase_duration','standardized_similarity_score']]
# purchase_duration 다른 변수 간의 상관관계 계산
correlation_matrix = data.corr(numeric_only=True)

# purchase_duration 다른 변수 간의 상관관계 추출
view_before_purchase_corr = correlation_matrix['purchase_duration'].sort_values(ascending=False)
view_before_purchase_corr


purchase_duration                1.000000
talk_duration                    0.748192
talk_user_count                  0.399126
price                            0.140872
image_count                      0.040145
des_length                       0.033453
flag_used                        0.000435
standardized_similarity_score   -0.009030
sum_score                       -0.086514
flag_taekpo                     -0.210060
Name: purchase_duration, dtype: float64

In [70]:
import statsmodels.api as sm
keywords=['price','flag_taekpo','image_count','des_length',
           'sum_score','standardized_similarity_score']
X = data[keywords]
y = data['purchase_duration']

# 상수 항 추가
X = sm.add_constant(X)

# 회귀 모델 적합
model = sm.OLS(y, X).fit(cov_type='HC3')


# 회귀 모델 요약 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      purchase_duration   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     2.959
Date:                Wed, 24 Jul 2024   Prob (F-statistic):            0.00993
Time:                        20:43:27   Log-Likelihood:                -891.10
No. Observations:                 127   AIC:                             1796.
Df Residuals:                     120   BIC:                             1816.
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         